In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
# pop=pd.read_csv('./US_2001_data/population_drought.csv')
# area=pd.read_csv('./US_2001_data/area_drought.csv')
# dsci=pd.read_csv('./US_2001_data/dsci_drought.csv')

In [3]:
from google.colab import files
uploaded=files.upload()


Saving abbrevs.csv to abbrevs (4).csv
Saving dsci_drought.csv to dsci_drought (4).csv
Saving area_drought.csv to area_drought (4).csv
Saving population_drought.csv to population_drought (4).csv


In [4]:
 import io
 pop=pd.read_csv(io.BytesIO(uploaded['population_drought.csv']))
 area=pd.read_csv(io.BytesIO(uploaded['area_drought.csv']))
 dsci=pd.read_csv(io.BytesIO(uploaded['dsci_drought.csv']))

In [5]:
pop['pop_none']=pop['None']
pop['pop_d0']=pop['D0']
pop['pop_d1']=pop['D1']
pop['pop_d2']=pop['D2']
pop['pop_d3']=pop['D3']
pop['pop_d4']=pop['D4']

pop.head()

,MapDate,StateAbbreviation,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4
0,20211221,AK,"709,629.99",0.00,0.00,0.00,0.00,0.00,2021-12-21,2021-12-27,2,"709,629.99",0.00,0.00,0.00,0.00,0.00
1,20211214,AK,"709,629.99",0.00,0.00,0.00,0.00,0.00,2021-12-14,2021-12-20,2,"709,629.99",0.00,0.00,0.00,0.00,0.00
2,20211207,AK,"709,629.99",0.00,0.00,0.00,0.00,0.00,2021-12-07,2021-12-13,2,"709,629.99",0.00,0.00,0.00,0.00,0.00
3,20211130,AK,"709,629.99",0.00,0.00,0.00,0.00,0.00,2021-11-30,2021-12-06,2,"709,629.99",0.00,0.00,0.00,0.00,0.00
4,20211123,AK,"709,629.99",0.00,0.00,0.00,0.00,0.00,2021-11-23,2021-11-29,2,"709,629.99",0.00,0.00,0.00,0.00,0.00


In [6]:
pop.drop(pop.iloc[:,:11], inplace = True, axis=1)
pop.head()

,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4
0,"709,629.99",0.00,0.00,0.00,0.00,0.00
1,"709,629.99",0.00,0.00,0.00,0.00,0.00
2,"709,629.99",0.00,0.00,0.00,0.00,0.00
3,"709,629.99",0.00,0.00,0.00,0.00,0.00
4,"709,629.99",0.00,0.00,0.00,0.00,0.00


In [7]:
area['area_none']=area['None']
area['area_d0']=area['D0']
area['area_d1']=area['D1']
area['area_d2']=area['D2']
area['area_d3']=area['D3']
area['area_d4']=area['D4']

area.head()

,MapDate,StateAbbreviation,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID,area_none,area_d0,area_d1,area_d2,area_d3,area_d4
0,20211221,AK,"582,568.29",0.00,0.00,0.00,0.00,0.00,2021-12-21,2021-12-27,2,"582,568.29",0.00,0.00,0.00,0.00,0.00
1,20211214,AK,"582,568.29",0.00,0.00,0.00,0.00,0.00,2021-12-14,2021-12-20,2,"582,568.29",0.00,0.00,0.00,0.00,0.00
2,20211207,AK,"582,568.29",0.00,0.00,0.00,0.00,0.00,2021-12-07,2021-12-13,2,"582,568.29",0.00,0.00,0.00,0.00,0.00
3,20211130,AK,"582,568.29",0.00,0.00,0.00,0.00,0.00,2021-11-30,2021-12-06,2,"582,568.29",0.00,0.00,0.00,0.00,0.00
4,20211123,AK,"582,568.29",0.00,0.00,0.00,0.00,0.00,2021-11-23,2021-11-29,2,"582,568.29",0.00,0.00,0.00,0.00,0.00


In [8]:
area.drop(columns=['MapDate','None','D0','D1','D2','D3','D4','ValidEnd'],inplace=True)
area.head()

,StateAbbreviation,ValidStart,StatisticFormatID,area_none,area_d0,area_d1,area_d2,area_d3,area_d4
0,AK,2021-12-21,2,"582,568.29",0.00,0.00,0.00,0.00,0.00
1,AK,2021-12-14,2,"582,568.29",0.00,0.00,0.00,0.00,0.00
2,AK,2021-12-07,2,"582,568.29",0.00,0.00,0.00,0.00,0.00
3,AK,2021-11-30,2,"582,568.29",0.00,0.00,0.00,0.00,0.00
4,AK,2021-11-23,2,"582,568.29",0.00,0.00,0.00,0.00,0.00


In [9]:
pop_area=pd.concat([area,pop], axis=1)

pop_area.head()

,StateAbbreviation,ValidStart,StatisticFormatID,area_none,area_d0,area_d1,area_d2,area_d3,area_d4,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4
0,AK,2021-12-21,2,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00
1,AK,2021-12-14,2,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00
2,AK,2021-12-07,2,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00
3,AK,2021-11-30,2,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00
4,AK,2021-11-23,2,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00


In [10]:
dsci

,Name,MapDate,DSCI
0,Alabama,20011225,31
1,Alabama,20020101,39
2,Alabama,20020108,49
3,Alabama,20020115,43
4,Alabama,20020122,31
...,...,...,...
54283,Wyoming,20211123,268
54284,Wyoming,20211130,278
54285,Wyoming,20211207,279
54286,Wyoming,20211214,274


In [11]:
abbrevs=pd.read_csv(io.BytesIO(uploaded['abbrevs.csv']))

abbrevs.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [12]:
abbrevs.shape

(51, 3)

In [13]:
pop_area_abbrevs=pd.merge(pop_area, abbrevs, how='outer',
                         left_on='StateAbbreviation', right_on='Code')
pop_area_abbrevs=pop_area_abbrevs.drop(columns='Code',axis=1)
pop_area_abbrevs.head()

#https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html

,StateAbbreviation,ValidStart,StatisticFormatID,area_none,area_d0,area_d1,area_d2,area_d3,area_d4,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4,State,Abbrev
0,AK,2021-12-21,2,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska,Alaska
1,AK,2021-12-14,2,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska,Alaska
2,AK,2021-12-07,2,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska,Alaska
3,AK,2021-11-30,2,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska,Alaska
4,AK,2021-11-23,2,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska,Alaska


In [14]:
pop_area_abbrevs.tail()

,StateAbbreviation,ValidStart,StatisticFormatID,area_none,area_d0,area_d1,area_d2,area_d3,area_d4,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4,State,Abbrev
54283,WY,2002-01-22,2,0.00,"11,417.07","24,322.47","34,644.88","27,414.20",0.00,0.00,"80,609.66","150,380.57","229,476.07","103,159.70",0.00,Wyoming,Wyo.
54284,WY,2002-01-15,2,0.00,"11,999.29","24,070.12","34,004.78","27,724.43",0.00,0.00,"89,323.78","144,270.76","225,615.99","104,415.48",0.00,Wyoming,Wyo.
54285,WY,2002-01-08,2,0.00,"11,479.21","24,336.02","34,188.91","27,794.48",0.00,0.00,"84,057.14","148,429.17","226,488.65","104,651.05",0.00,Wyoming,Wyo.
54286,WY,2002-01-01,2,"2,957.24","22,697.96","18,087.99","26,237.41","27,818.02",0.00,"60,774.68","124,529.41","89,923.30","183,545.63","104,852.98",0.00,Wyoming,Wyo.
54287,WY,2001-12-25,2,"2,426.71","23,258.74","17,476.79","26,850.06","27,786.33",0.00,"54,549.99","130,732.86","86,477.77","187,090.33","104,775.05",0.00,Wyoming,Wyo.


In [15]:
pop_area_abbrevs.isnull().sum()

StateAbbreviation       0
ValidStart              0
StatisticFormatID       0
area_none               0
area_d0                 0
area_d1                 0
area_d2                 0
area_d3                 0
area_d4                 0
pop_none                0
pop_d0                  0
pop_d1                  0
pop_d2                  0
pop_d3                  0
pop_d4                  0
State                1044
Abbrev               1044
dtype: int64

In [16]:
pop_area_abbrevs.loc[pop_area_abbrevs['State'].isnull(),'StateAbbreviation'].unique()

array(['PR'], dtype=object)

In [17]:
pop_area_abbrevs.loc[pop_area_abbrevs['StateAbbreviation']=='PR','State']= 'Puerto Rico'

pop_area_abbrevs.loc[pop_area_abbrevs['StateAbbreviation']=='PR','Abbrev']='PR'

pop_area_abbrevs.isnull().sum()

StateAbbreviation    0
ValidStart           0
StatisticFormatID    0
area_none            0
area_d0              0
area_d1              0
area_d2              0
area_d3              0
area_d4              0
pop_none             0
pop_d0               0
pop_d1               0
pop_d2               0
pop_d3               0
pop_d4               0
State                0
Abbrev               0
dtype: int64

In [18]:
pop_area_abbrevs.drop(columns=['StatisticFormatID','Abbrev'],inplace=True)
pop_area_abbrevs.head()

,StateAbbreviation,ValidStart,area_none,area_d0,area_d1,area_d2,area_d3,area_d4,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4,State
0,AK,2021-12-21,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska
1,AK,2021-12-14,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska
2,AK,2021-12-07,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska
3,AK,2021-11-30,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska
4,AK,2021-11-23,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska


In [19]:
pop_area_abbrevs.drop(columns=['StateAbbreviation'],inplace=True)
pop_area_abbrevs.head()

,ValidStart,area_none,area_d0,area_d1,area_d2,area_d3,area_d4,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4,State
0,2021-12-21,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska
1,2021-12-14,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska
2,2021-12-07,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska
3,2021-11-30,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska
4,2021-11-23,"582,568.29",0.00,0.00,0.00,0.00,0.00,"709,629.99",0.00,0.00,0.00,0.00,0.00,Alaska


In [20]:
cols=[i for i in pop_area_abbrevs.columns if i not in ['ValidStart','State']]
for col in cols:
        pop_area_abbrevs[col]=[float(str(i).replace(",",""))for i in pop_area_abbrevs[col]]
#https://stackoverflow.com/questions/44602139/pandas-convert-all-column-from-string-to-number-except-two
#https://stackoverflow.com/questions/8420143/valueerror-could-not-convert-string-to-float-id     

In [21]:
pop_area_abbrevs.dtypes

ValidStart     object
area_none     float64
area_d0       float64
area_d1       float64
area_d2       float64
area_d3       float64
area_d4       float64
pop_none      float64
pop_d0        float64
pop_d1        float64
pop_d2        float64
pop_d3        float64
pop_d4        float64
State          object
dtype: object

In [22]:
dsci.drop(columns='MapDate',inplace=True)


In [23]:
dsci.rename(columns={'Name':'State'},inplace=True)

dsci.head()

,State,DSCI
0,Alabama,31
1,Alabama,39
2,Alabama,49
3,Alabama,43
4,Alabama,31


In [24]:
dsci.shape

(54288, 2)

In [25]:
dsci.tail()

,State,DSCI
54283,Wyoming,268
54284,Wyoming,278
54285,Wyoming,279
54286,Wyoming,274
54287,Wyoming,275


In [26]:
final_merged=pd.merge(dsci,pop_area_abbrevs, on ='State')
final_merged.head()
#https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html

,State,DSCI,ValidStart,area_none,area_d0,area_d1,area_d2,area_d3,area_d4,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4
0,Alabama,31,2021-12-21,34731.33,15357.43,1515.59,0.0,0.0,0.0,3871252.46,882211.41,26272.15,0.0,0.0,0.0
1,Alabama,31,2021-12-14,28091.42,22353.31,1159.61,0.0,0.0,0.0,3429642.99,1331612.85,18480.18,0.0,0.0,0.0
2,Alabama,31,2021-12-07,24009.42,26435.31,1159.61,0.0,0.0,0.0,3011792.40,1749463.44,18480.18,0.0,0.0,0.0
3,Alabama,31,2021-11-30,26482.82,25121.53,0.00,0.0,0.0,0.0,3153556.89,1626179.13,0.00,0.0,0.0,0.0
4,Alabama,31,2021-11-23,37734.50,13869.85,0.00,0.0,0.0,0.0,3917434.59,862301.43,0.00,0.0,0.0,0.0


In [27]:
final_merged.tail()

,State,DSCI,ValidStart,area_none,area_d0,area_d1,area_d2,area_d3,area_d4,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4
56676667,Wyoming,275,2002-01-22,0.00,11417.07,24322.47,34644.88,27414.20,0.0,0.00,80609.66,150380.57,229476.07,103159.70,0.0
56676668,Wyoming,275,2002-01-15,0.00,11999.29,24070.12,34004.78,27724.43,0.0,0.00,89323.78,144270.76,225615.99,104415.48,0.0
56676669,Wyoming,275,2002-01-08,0.00,11479.21,24336.02,34188.91,27794.48,0.0,0.00,84057.14,148429.17,226488.65,104651.05,0.0
56676670,Wyoming,275,2002-01-01,2957.24,22697.96,18087.99,26237.41,27818.02,0.0,60774.68,124529.41,89923.30,183545.63,104852.98,0.0
56676671,Wyoming,275,2001-12-25,2426.71,23258.74,17476.79,26850.06,27786.33,0.0,54549.99,130732.86,86477.77,187090.33,104775.05,0.0


In [28]:
#final_merged.to_csv('./US_2001_data/us_drought_cleaned.csv')

In [29]:
final_merged.shape

(56676672, 15)

In [30]:
final_merged['DSCI'].value_counts()

0      16572456
1        772560
3        686952
2        676512
5        580464
         ...   
442        1044
475        1044
424        1044
481        1044
482        1044
Name: DSCI, Length: 476, dtype: int64

In [31]:
final_merged['ValidStart']=pd.to_datetime(final_merged['ValidStart'])

In [32]:
final_merged.dtypes

State                 object
DSCI                   int64
ValidStart    datetime64[ns]
area_none            float64
area_d0              float64
area_d1              float64
area_d2              float64
area_d3              float64
area_d4              float64
pop_none             float64
pop_d0               float64
pop_d1               float64
pop_d2               float64
pop_d3               float64
pop_d4               float64
dtype: object

In [33]:
final_merged=final_merged.set_index('ValidStart')

final_merged.head()

,State,DSCI,area_none,area_d0,area_d1,area_d2,area_d3,area_d4,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4
ValidStart,,,,,,,,,,,,,,
2021-12-21,Alabama,31,34731.33,15357.43,1515.59,0.0,0.0,0.0,3871252.46,882211.41,26272.15,0.0,0.0,0.0
2021-12-14,Alabama,31,28091.42,22353.31,1159.61,0.0,0.0,0.0,3429642.99,1331612.85,18480.18,0.0,0.0,0.0
2021-12-07,Alabama,31,24009.42,26435.31,1159.61,0.0,0.0,0.0,3011792.40,1749463.44,18480.18,0.0,0.0,0.0
2021-11-30,Alabama,31,26482.82,25121.53,0.00,0.0,0.0,0.0,3153556.89,1626179.13,0.00,0.0,0.0,0.0
2021-11-23,Alabama,31,37734.50,13869.85,0.00,0.0,0.0,0.0,3917434.59,862301.43,0.00,0.0,0.0,0.0


In [34]:
final_merged.head(30)

,State,DSCI,area_none,area_d0,area_d1,area_d2,area_d3,area_d4,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4
ValidStart,,,,,,,,,,,,,,
2021-12-21,Alabama,31,34731.33,15357.43,1515.59,0.0,0.0,0.0,3871252.46,882211.41,26272.15,0.0,0.0,0.0
2021-12-14,Alabama,31,28091.42,22353.31,1159.61,0.0,0.0,0.0,3429642.99,1331612.85,18480.18,0.0,0.0,0.0
2021-12-07,Alabama,31,24009.42,26435.31,1159.61,0.0,0.0,0.0,3011792.40,1749463.44,18480.18,0.0,0.0,0.0
2021-11-30,Alabama,31,26482.82,25121.53,0.00,0.0,0.0,0.0,3153556.89,1626179.13,0.00,0.0,0.0,0.0
2021-11-23,Alabama,31,37734.50,13869.85,0.00,0.0,0.0,0.0,3917434.59,862301.43,0.00,0.0,0.0,0.0
2021-11-16,Alabama,31,50440.75,1163.60,0.00,0.0,0.0,0.0,4752055.32,27680.71,0.00,0.0,0.0,0.0
2021-11-09,Alabama,31,51604.35,0.00,0.00,0.0,0.0,0.0,4779736.02,0.00,0.00,0.0,0.0,0.0
2021-11-02,Alabama,31,51604.35,0.00,0.00,0.0,0.0,0.0,4779736.02,0.00,0.00,0.0,0.0,0.0
2021-10-26,Alabama,31,51604.35,0.00,0.00,0.0,0.0,0.0,4779736.02,0.00,0.00,0.0,0.0,0.0


In [35]:
final_merged_smaller=final_merged[3::8]
final_merged_smaller.shape

(7084584, 14)

In [39]:
final_merged_smaller.drop(columns='State',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [40]:
features = [col for col in final_merged_smaller.columns if not col == 'DSCI']

X=final_merged_smaller[features]

y=final_merged_smaller[['DSCI']].values

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=False)

X_train.head()

,area_none,area_d0,area_d1,area_d2,area_d3,area_d4,pop_none,pop_d0,pop_d1,pop_d2,pop_d3,pop_d4
ValidStart,,,,,,,,,,,,
2021-11-30,26482.82,25121.53,0.0,0.0,0.0,0.0,3153556.89,1626179.13,0.0,0.0,0.0,0.0
2021-10-05,51604.35,0.00,0.0,0.0,0.0,0.0,4779736.02,0.00,0.0,0.0,0.0,0.0
2021-08-10,51604.35,0.00,0.0,0.0,0.0,0.0,4779736.02,0.00,0.0,0.0,0.0,0.0
2021-06-15,51604.35,0.00,0.0,0.0,0.0,0.0,4779736.02,0.00,0.0,0.0,0.0,0.0
2021-04-20,49321.64,2282.71,0.0,0.0,0.0,0.0,4668042.51,111693.51,0.0,0.0,0.0,0.0


In [42]:
ss= StandardScaler()

X_train_sc=ss.fit_transform(X_train)

X_test_sc=ss.transform(X_test)

In [43]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [44]:
train_sequences= TimeseriesGenerator(X_train_sc, y_train, length=4, batch_size=64)

In [45]:
batch_x,batch_y=train_sequences[0]

In [46]:
batch_x.shape

(64, 4, 12)

In [47]:
batch_y.shape

(64, 1)

In [48]:
test_sequences=TimeseriesGenerator(X_test_sc, y_test, length =4, batch_size=64)

In [49]:
y[:5]

array([[31],
       [31],
       [31],
       [31],
       [31]])

In [52]:
input_shape=batch_x.shape[1:]

model = Sequential()

model.add(GRU(8, input_shape = input_shape, return_sequences=True))
model.add(GRU(8, return_sequences=False))
model.add(Dense(4,activation = 'relu'))
model.add(Dense(4,activation = 'relu'))
model.add(Dense(1,activation ='linear'))

model.compile(
    loss='mse', optimizer = Adam(lr=.0005),
    metrics= ['acc']
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [53]:
history= model.fit(train_sequences, validation_data=test_sequences, epochs= 50)

Epoch 1/50
83023/83023 [==============================] - 534s 6ms/step - loss: 8540.8252 - acc: 0.0137 - val_loss: 8613.7354 - val_acc: 0.0147
Epoch 2/50
83023/83023 [==============================] - 506s 6ms/step - loss: 8036.5596 - acc: 0.0133 - val_loss: 8772.4844 - val_acc: 0.0147
Epoch 3/50
83023/83023 [==============================] - 507s 6ms/step - loss: 8015.2842 - acc: 0.0133 - val_loss: 8707.6641 - val_acc: 0.0147
Epoch 4/50
83023/83023 [==============================] - 508s 6ms/step - loss: 7993.3901 - acc: 0.0133 - val_loss: 8772.6797 - val_acc: 0.0147
Epoch 5/50
83023/83023 [==============================] - 506s 6ms/step - loss: 7970.1436 - acc: 0.0133 - val_loss: 8770.1504 - val_acc: 0.0147
Epoch 6/50
83023/83023 [==============================] - 503s 6ms/step - loss: 7949.1538 - acc: 0.0133 - val_loss: 8739.9043 - val_acc: 0.0147
Epoch 7/50
83023/83023 [==============================] - 501s 6ms/step - loss: 7927.0508 - acc: 0.0133 - val_loss: 8791.9541 - val_acc: